In [39]:
import pandas as pd
import chardet
import json
import re
import os
import openpyxl

In [72]:
#carga manual de dataset
#TAREA: automatizarlo
dataset1 = 'Datasets_relevamiento_precios/precios_semana_20200413.csv' #Csv precio_semana
dataset2 = 'Datasets_relevamiento_precios/precios_semana_20200503.json' #json precio_semana
dataset3 = 'Datasets_relevamiento_precios/precios_semana_20200518.txt'#txt precio_semana
#dataset4 = 'Datasets_relevamiento_precios/precios_semanas_20200419_20200426.xlsx' #xls precio_semana
dataset5 = 'Datasets_relevamiento_precios/producto.parquet' #parquet producto
dataset6 = 'Datasets_relevamiento_precios/sucursal.csv' #csv sucursal



In [73]:
#abro el archivo y veo el encoding - 
#TAREA: buscar si hay otra forma autom. de hacerlo
def detect_encoding(dataset):
    with open(dataset, 'rb') as f:
        enc = chardet.detect(f.read())
    return enc['encoding']         

In [74]:
#dataset precio semana 1
precio_semana1 = pd.read_csv(dataset1, delimiter=",", encoding=detect_encoding(dataset1))

In [83]:
precio_semana1

,precio,producto_id,sucursal_id
0,29.90,0000000001663,2-1-014
1,29.90,0000000002288,2-1-032
2,39.90,0000000002288,2-1-096
3,499.99,0000000205870,9-1-686
4,519.99,0000000205870,9-2-248
...,...,...,...
472161,139.99,9569753142128,25-1-1
472162,34.99,9795403001143,25-1-1
472163,312.50,9990385651922,5-1-3
472164,312.50,9990385651939,5-1-3


In [75]:
#dataset precio semana 2
precio_semana2 = pd.read_json(dataset2)

In [79]:
precio_semana2.head()

,precio,producto_id,sucursal_id
0,29.9,0000000002288,2-1-187
1,39.9,0000000002288,2-3-247
2,499.99,0000000205870,9-1-685
3,539.99,0000000205870,9-2-22
4,519.99,0000000205870,9-2-59


In [76]:
#dataset precio semana 3
#pd.read_fwf(dataset4)
precio_semana3 = pd.read_csv(dataset3, delimiter="|", encoding=detect_encoding(dataset3))


In [80]:
precio_semana3.head()

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430


In [ ]:
#dataset precio semana 4
precios_xlsx = pd.read_excel('Datasets_relevamiento_precios/precios_semanas_20200419_20200426.xlsx', sheet_name=None)

In [81]:
# uso concat para hacer una sola columna del dataframe 
precio_semana4 = pd.concat(precios_xlsx, ignore_index=True)

In [82]:
precio_semana4

,precio,sucursal_id,producto_id
0,399.00,2-1-092,2288.0
1,299.00,2-1-206,2288.0
2,399.00,2-2-241,2288.0
3,49999.00,9-1-430,205870.0
4,53999.00,9-2-4,205870.0
...,...,...,...
937447,139.99,2001-01-25 00:00:00,9569753142128
937448,34.99,2001-01-25 00:00:00,9795403001143
937449,312.50,2003-01-05 00:00:00,9990385651922
937450,312.50,2003-01-05 00:00:00,9990385651939


In [181]:
#dataset producto
producto = pd.read_parquet(dataset5, engine='pyarrow')

In [117]:
#tamaño de la tabla producto.shape (72038, 7)
#la tabla no tiene productos duplicados --> producto.drop_duplicates(inplace=True)
#shape despues de borrar duplicados es la misma #tamaño de la tabla (72038, 7)

In [170]:
#tamaño de la tabla
producto.shape

(72038, 7)

In [118]:
#tamaño de la tabla
producto.shape

(72038, 7)

In [107]:
#ver valores nulos 
producto.isnull().sum() #mismo resultado si uso .isna 

id                  0
marca               2
nombre              2
presentacion        2
categoria1      72034
categoria2      72034
categoria3      72034
dtype: int64

In [ ]:
#las columnas últimas 3 columnas tienen un % muy bajo de valores y es muy difícil inferir cuales serían los datos faltantes sólo en base a la información que tenemos.
#Elimino las columnas y si fuera un caso real, le muestro los siguientes porcentajes al cliente para su consideración. 
# Además le preguntaría si esa info es relevante y de serlo, propondría armar una tabla de categorías (id producto y categoría)

In [139]:
#guardo el total en una variable
total = producto.shape[0]


In [175]:
producto.categoria1.isnull().sum()

72034

In [176]:
nulos_cat1 = round((72034*100/total), 2)
nulos_cat2 = round((72034*100/total), 2)
nulos_cat3 = round((72034*100/total), 2)
print("total de faltantes categoría 1:", nulos_cat1, '%')
print("total de faltantes categoría 2:", nulos_cat3, '%')
print("total de faltantes categoría 3:", nulos_cat2, '%')

total de faltantes categoría 1: 99.99 %
total de faltantes categoría 2: 99.99 %
total de faltantes categoría 3: 99.99 %


In [182]:
#dropeo las ultimas columnas
producto.drop(['categoria1', 'categoria2', 'categoria3'], axis=1, inplace=True)

In [114]:
#veo cuales son los porductos sin 'nombre'
producto[producto['nombre'].isnull()]
#coinciden con los 2 null de marca

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
53619,7798024299045,None,None,None,Almacén,Aceites,Aerosol
55798,7798061190213,None,None,None,Almacén,Aceites,Oliva


In [183]:
#seteo el valor de datos faltantes en "sin dato"
producto['nombre'].fillna("Sin Dato", inplace = True)
producto['marca'].fillna("Sin Dato", inplace = True)
producto['presentacion'].fillna("Sin Dato", inplace = True)

In [155]:
producto['id'].value_counts().sum #no hay valores repetidos en en id :) se puede usar como clave primario de ser necesario

72038

In [190]:
#elimino los números y unidades del nombre ya que parece un error de carga 
producto.nombre = producto.nombre.astype(str).apply(lambda x: x.strip('0123456789UnKgGr '))

In [191]:
producto

,id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima,1.0 un
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima,1.0 un
2,0000000205870,SIN MARCA,Ojo de Bife,1.0 kg
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito,1.0 kg
4,0000000205955,SIN MARCA,Chiquizuela Novillito,1.0 kg
...,...,...,...,...
72033,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Sitas,500.0 gr
72034,9795403001143,MAYO,Mini Pizzetas Mayo,12.0 un
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana,50.0 gr
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana,50.0 gr


In [194]:
#dataset sucursal
sucursal = pd.read_csv(dataset6, delimiter=",", encoding=detect_encoding(dataset6))

In [196]:
sucursal.head()

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado
